In [99]:
from datetime import datetime as dt
from pathlib import Path
from collections import defaultdict

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

LOG_DIR = "/home/engineer/fltk-testbed/logging"
EXP_NAME = "exp_8"
CREATE_LOG = f"{LOG_DIR}/{EXP_NAME}/create.log"
START_LOG = f"{LOG_DIR}/{EXP_NAME}/start.log"
STOP_LOG = f"{LOG_DIR}/{EXP_NAME}/stop.log"

In [100]:
logs = [CREATE_LOG, START_LOG, STOP_LOG]
containers = defaultdict(dict)

for log in logs:
    with open(log, "r") as f:
        while line := f.readline():
            split = line.rstrip().split()
            key = Path(log).stem
            value = dt.strptime(f"{split[0]} {split[1]}", "%m-%d-%Y %H:%M:%S")
            containers[split[2]] |= {key: value}

In [101]:
for c in containers:
    container = containers[c]
    assert container["create"] < container["start"], f"{c} created after started"
    assert container["start"] < container["stop"], f"{c} started after stopped"
    container["queue"] = (container["start"] - container["create"]).total_seconds()
    container["service"] = (container["stop"] - container["start"]).total_seconds()
    container["response"] = (container["stop"] - container["create"]).total_seconds()

In [102]:
queues = []
services = []
responses = []

for c in containers:
    container = containers[c]
    queues.append(container["queue"] / 60)
    services.append(container["service"] / 60)
    responses.append(container["response"] / 60)


In [103]:
def plot(data, containers, label):
    # fig, ax = plt.subplots(figsize=(15, 5), tight_layout=True)
    # ax.axhline(
    #     np.mean(data),
    #     xmin=0.05,
    #     xmax=0.95,
    #     linestyle="--",
    #     color="r",
    #     alpha=0.5,
    #     label="Mean",
    # )
    x = list(range(1, len(containers) + 1))
    # ax.plot(x, data, "-o", label=label)
    # ax.legend()
    # ax.set_xticks(x)
    # ax.set_xlabel("Job")
    # ax.set_ylabel("Time (min)")
    # ax.set_title(f"{label} for Jobs")
    mean = np.mean(data)
    std = np.std(data)
    # fig.text(
    #     0.5,
    #     -0.07,
    #     f"mean: {mean.round(2)}\nstd: {std.round(2)}",
    #     ha="center",
    #     bbox=dict(facecolor="none", edgecolor="lightgray", boxstyle="round"),
    # )
    # fig.autofmt_xdate()
    # fig = px.line(
    #     x=x,
    #     y=data,
    #     markers=True,
    #     labels={
    #         "x": "Job",
    #         "y": "Time (min)",
    #     },
    # )
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=data, mode="lines+markers", name=label))
    fig.add_hline(
        y=np.mean(data),
        line_dash="dash",
        line_color="red",
        annotation_text=f"Mean: {mean.round(2)}",
        annotation_position="top right",
    )

    fig.add_annotation(
        text=f"Std: {std.round(2)}",
        xref="paper",
        yref="paper",
        x=1,
        y=1,
        showarrow=False,
    )

    fig.update_layout(
        xaxis_title="Job",
        yaxis_title="Time (min)",
        xaxis=dict(tickvals=x),
    )
    fig.update_xaxes(tickangle=90)

    fig.show()
    fig.write_image(f"./figures/queue_{label}.svg")

    return mean, std

In [104]:
mean_queue, std_queue = plot(queues, containers, "Queue Time")
mean_service, std_service = plot(services, containers, "Service Time")
mean_response, std_response = plot(responses, containers, "Response Time")

print(f"Mean Queue Time: {mean_queue}")
print(f"Std Queue Time: {std_queue}")
print(f"Mean Service Time: {mean_service}")
print(f"Std Service Time: {std_service}")
print(f"Mean Response Time: {mean_response}")
print(f"Std Response Time: {std_response}")

Mean Queue Time: 0.5405555555555556
Std Queue Time: 0.18502669143453299
Mean Service Time: 0.6599999999999999
Std Service Time: 0.048035480713816185
Mean Response Time: 1.200555555555556
Std Response Time: 0.19614258482095853


In [105]:
# Compare all queues with Mean Response Time
LAMBDA = 0.5
MU = 1 / mean_service
RHO = LAMBDA / MU
print(RHO)
print(MU)

# M/M/1
mm1_mean_response = 1 / (MU - LAMBDA)
# print((RHO / (1-RHO))*(1/MU)+1/MU)

# M/D/1
md1_mean_response = (RHO + (1 / 2) * (RHO**2 / (1 - RHO))) / LAMBDA

# M/G/1
mg1_mean_response = mean_service + (std_service**2 / ((2 * mean_service) * (1 - RHO)))

print(f"M/M/1 Mean Response Time: {mm1_mean_response}")
print(f"M/D/1 Mean Response Time: {md1_mean_response}")
print(f"M/G/1 Mean Response Time: {mg1_mean_response}")

0.32999999999999996
1.5151515151515154
M/M/1 Mean Response Time: 0.9850746268656715
M/D/1 Mean Response Time: 0.8225373134328356
M/G/1 Mean Response Time: 0.6626090088279142
